In [1]:
import os; import arcpy; import numpy as np
#Set Coordinate System
spatialRef    = arcpy.SpatialReference(4326) #WGS1984
spatialRefPrj = arcpy.SpatialReference(32662) #WGS_1984_Plate_Carre

climateVariable = 'HC' #T or HC
#Set file names and locations
data_dir = r'Z:\Desktop\HoloceneHydroclimate\DataSummary'
dataname = 'proxy'+climateVariable
datapath = os.path.join(data_dir,dataname+'.csv')
XFieldName = 'Lon'; YFieldName = 'Lat'
datanameProject = dataname+"_project"

In [2]:
#Load and project spatial data from csv file
arcpy.env.overwriteOutput = True
arcpy.management.XYTableToPoint(datapath,dataname,XFieldName,YFieldName,coordinate_system=spatialRef)
arcpy.management.Project(dataname,datanameProject,spatialRefPrj)

<Result 'Z:\\Desktop\\HoloceneHydroclimate\\AnalysisCode\\ArcGIS\\ClusterProject\\ClusterProject.gdb\\proxyHC_project'>

In [6]:
if climateVariable == 'HC':
    ClusterField = ['LateSlope','TotalSlope'] #Attributes to weight clustering by 
    #ClusterField = ['bin6ka','bin3ka']
    spatialWeight = 6
elif climateVariable == 'T':
    ClusterField = ['LateSlope','EarlySlope'] #Attributes to weight clustering by 
    #ClusterField = ['bin6ka','bin3ka']
    spatialWeight = 8


#Cluster sites based on geographic proximity and attribute values
arcpy.env.overwriteOutput = True
ClusterSelect= arcpy.SelectLayerByAttribute_management(datanameProject,"NEW_SELECTION","Lat>-60") #Seperate antarctic records from analysis
ClusterCount = arcpy.GetCount_management(ClusterSelect).getOutput(0) #Get count of records
print(ClusterCount)
ClusterCountMin = int(np.nanmax([2,int(float(ClusterCount)*0.02)]))
ClusterCountMin = 8
print(ClusterCountMin)
SpatialWeights = arcpy.stats.GenerateSpatialWeightsMatrix(ClusterSelect,
        "Field1",r"Z:\Desktop\HoloceneHydroclimate\AnalysisCode\ArcGIS\ClusterSpatialWeights.swm", "K_NEAREST_NEIGHBORS",
        "EUCLIDEAN", 2, None,spatialWeight, "ROW_STANDARDIZATION", None, None, '', None, None)
arcpy.stats.SpatiallyConstrainedMultivariateClustering(ClusterSelect,dataname+"_Cluster",
    ClusterField,"NUM_FEATURES", None,ClusterCountMin,None,None,
    "GET_SPATIAL_WEIGHTS_FROM_FILE", SpatialWeights, 0, None) #"TRIMMED_DELAUNAY_TRIANGULATION"


687
8


id,value
0,Z:\Desktop\HoloceneHydroclimate\AnalysisCode\ArcGIS\ClusterProject\ClusterProject.gdb\proxyHC_Cluster
1,
